# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv") #added the ../

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,18.78,80,23,6.56,ZA,1711994422
1,1,kaspiyskiy,45.3929,47.3707,9.70,66,13,4.31,RU,1711994422
2,2,utrik,11.2278,169.8474,27.15,69,100,8.51,MH,1711994422
3,3,stanley,54.8680,-1.6985,7.06,96,75,2.57,GB,1711994380
4,4,bethel,41.3712,-73.4140,11.68,57,100,1.39,US,1711994423


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display #suppress the output of a code cell. it allows you to access and manipulate the captured output programmatically.

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points( #taking info from city_data_df and using hvplot for plotting
    "Lng",
    "Lat",
    geo=True, #specifies the plot should be rendered on a map (geographical coordenate system)
    size = "Humidity", #humidity values determine the size of the points
    tiles = "OSM", #there are a variety of map display options. OSM is a generally used one.
    frame_width = 800,
    frame_height = 600,
    color = "City", #specifies that each city gets a different color
    alpha = 0.5, #transparency level of the dots themselves
)

# Display the map

map_plot_1



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Temperature, humidity and cloudiness parameters are defined as filters for the dataframe:
max_temp_range = (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30)
humidity_condition = city_data_df['Humidity'] < 70
cloudiness_condition = city_data_df['Cloudiness'] < 50
ideal_cities_df = city_data_df[max_temp_range & humidity_condition & cloudiness_condition]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,umm kaddadah,13.6017,26.6876,27.37,9,15,5.21,SD,1711994427
29,29,udumalaippettai,10.5833,77.2500,26.05,66,17,2.49,IN,1711994429
36,36,zeribet el oued,34.6828,6.5111,25.26,22,24,9.20,DZ,1711994431
41,41,carnarvon,-24.8667,113.6333,26.11,69,31,6.75,AU,1711994408
51,51,brisas de zicatela,15.8369,-97.0419,29.94,58,0,4.38,MX,1711994434


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()  #copy is not strictly required, it just assures that the new dataframe is completely independent of the city_data_df one.

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
19,umm kaddadah,SD,13.6017,26.6876,9,
29,udumalaippettai,IN,10.5833,77.2500,66,
36,zeribet el oued,DZ,34.6828,6.5111,22,
41,carnarvon,AU,-24.8667,113.6333,69,
51,brisas de zicatela,MX,15.8369,-97.0419,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000 #10,000 meters from the coordinates
limit = 1
params = {
    "radius": radius,
    "categories" : "accommodation.hotel", #parameter for a hotel search
    "limit" : limit, #limit to 1 result
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"

    #while both filter and bias parameters help refine search results, "filter" is used to specify criteria that the 
    #results must satisfy, while "bias" is used to prioritize results based on geographical proximity.
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url , params=params) #calling the API using the base url and the specified params
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"] #takes the name from the dictionary and stores it as the Hotel Name in the dataframe.
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
umm kaddadah - nearest hotel: No hotel found
udumalaippettai - nearest hotel: Hotel anna Vasal
zeribet el oued - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
port hedland - nearest hotel: The Esplanade Hotel
alexandria - nearest hotel: Smoha Zahran Haus (7th floor)
tura - nearest hotel: Hotel Polo Orchid
tamanrasset - nearest hotel: فندق الأمان
coronel suarez - nearest hotel: La Casa Hotel Boutique
narsipatnam - nearest hotel: No hotel found
jinghong - nearest hotel: โรงแรมสืเบา
praya - nearest hotel: Casa Sodadi
yenangyaung - nearest hotel: No hotel found
rabigh - nearest hotel: S.A. TALKE Villa
qaryat suluq - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
kas - nearest hotel: KEKOVA OTEL
al burayqah - nearest hotel: فندق ماس ليبيا
badr hunayn - nearest hotel: No hotel found
najran - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
19,umm kaddadah,SD,13.6017,26.6876,9,No hotel found
29,udumalaippettai,IN,10.5833,77.2500,66,Hotel anna Vasal
36,zeribet el oued,DZ,34.6828,6.5111,22,No hotel found
41,carnarvon,AU,-24.8667,113.6333,69,No hotel found
51,brisas de zicatela,MX,15.8369,-97.0419,58,Casa de Olas
52,port hedland,AU,-20.3167,118.5667,61,The Esplanade Hotel
62,alexandria,EG,31.2156,29.9553,66,Smoha Zahran Haus (7th floor)
64,tura,IN,25.5198,90.2201,23,Hotel Polo Orchid
77,tamanrasset,DZ,22.7850,5.5228,13,فندق الأمان
101,coronel suarez,AR,-37.4547,-61.9334,19,La Casa Hotel Boutique


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True, #specifies the plot should be rendered on a map (geographical coordenate system)
    size = "Humidity", #humidity values determine the size of the points
    tiles = "OSM", #there are a variety of map display options. OSM is a generally used one.
    frame_width = 800,
    frame_height = 600,
    color = "City", #specifies that each city gets a different color
    alpha = 0.5, #transparency level of the dots themselves
    hover_cols=["Hotel Name", "Country"], #adding additional data in the hovering over the map points
)

# Display the map

map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)